<a href="https://colab.research.google.com/github/julialorrayne/Healthcare-Revenue-Operations-Dashboard/blob/main/Hospital_Database_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary Python Libraries

import csv
import random
import datetime


In [ ]:
# Configuration to control the volume of data

num_departments = 5

# Feel free to modify the below
num_doctors = 35
num_patients = 1200
num_appointments = 8000
num_diagnoses = 40
num_billing = 5200 # Only for completed appointments
num_appt_diag = 12000
num_departments = 5



In [ ]:

# Predefined values
departments = ["Cardiology", "Radiology", "Orthopedics", "Neurology", "Oncology"]
sex_values = ["M", "F"]
diagnosis_descs = [
    "Hypertension","Type 2 Diabetes","Asthma","Breast Cancer","Migraine",
    "Fracture","Osteoarthritis","Stroke","Influenza","COVID-19",
    "Back Pain","Anxiety","Depression","Allergic Rhinitis","Pneumonia",
    "Coronary Artery Disease","Heart Failure","Arrhythmia","Kidney Stones","UTI",
    "COPD","Tendonitis","Concussion","GERD","Peptic Ulcer",
    "Dermatitis","Otitis Media","Sinusitis","Thyroid Disorder","Anemia",
    "Obesity","High Cholesterol","Cataract","Glaucoma","Hepatitis",
    "Pancreatitis","Seizure Disorder","Lung Cancer","Leukemia","Arthralgia"
]

status_weights = [("Completed", 0.65), ("No-Show", 0.15), ("Cancelled", 0.10), ("Scheduled", 0.10)]

def weighted_status():
    r = random.random()
    cum = 0
    for s, w in status_weights:
        cum += w
        if r <= cum:
            return s
    return "Completed"

# Generate Departments
num_departments = len(departments)
department_records = [(i+1, dept) for i, dept in enumerate(departments)]

# Generate Doctors
doctor_first_names = [
    "James","Maria","John","Ana","Robert","Lucia","Michael","Sofia","David","Carla",
    "Daniel","Paula","Bruno","Camila","Pedro","Beatriz","Rafael","Mariana","Lucas","Fernanda",
    "Gabriel","Juliana","Andre","Patricia","Thiago","Larissa","Victor","Renata","Felipe","Aline",
    "Ricardo","Isabela","Eduardo","Leticia","Henrique","Bianca","Gustavo","Nathalia","Diego","Claudia"
]

doctor_last_names = [
    "Smith","Johnson","Williams","Brown","Jones","Garcia","Miller","Davis","Rodriguez","Martinez",
    "Hernandez","Lopez","Gonzalez","Wilson","Anderson","Thomas","Taylor","Moore","Jackson","Martin",
    "Lee","Perez","Thompson","White","Harris","Sanchez","Clark","Ramirez","Lewis","Robinson",
    "Walker","Young","Allen","King","Wright","Scott","Torres","Nguyen","Hill","Flores",
    "Silva","Santos","Costa","Almeida","Ribeiro","Gomes","Pereira","Carvalho","Araujo","Nascimento"
]

# Generate Doctors (unique full names)
name_pool = [(fn, ln) for fn in doctor_first_names for ln in doctor_last_names]
random.shuffle(name_pool)

doctor_records = []
for i in range(num_doctors):
    fn, ln = name_pool[i]
    doctor_records.append((101+i, fn, ln, random.randint(1, num_departments)))



# Generate Patients
patient_first_names = [
    "Lucas","Emma","Noah","Olivia","Liam","Ava","Mateo","Isabella","Ethan","Mia",
    "Ben","Chloe","Diego","Laura","Rafael","Beatriz","Daniel","Sophia","Henry","Camila"
]

patient_last_names = [
    "Rodriguez","Williams","Santos","Miller","Gonzalez","Costa","Taylor","Almeida",
    "Clark","Silva","Pereira","Gomes","Ribeiro","Martins","Araujo","Nascimento"
]

patient_records = []
for i in range(num_patients):
    dob = datetime.date(
        random.randint(1950, 2010),
        random.randint(1, 12),
        random.randint(1, 28)
    )

    insurance = random.choices(
        population=[None, "PlanA", "PlanB", "PlanC"],
        weights=[0.12, 0.35, 0.38, 0.15]
    )[0]

    patient_records.append((
        201+i,
        random.choice(patient_first_names),
        random.choice(patient_last_names),
        dob,
        random.choice(sex_values),
        f"555-{random.randint(100,999)}-{random.randint(1000,9999)}",
        f"patient{201+i}@example.com",
        insurance
    ))


# Generate Appointments
appointment_records = []
completed_appts = []
for i in range(num_appointments):
    year = random.choices([2023, 2024, 2025], weights=[0.25, 0.35, 0.40])[0]
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    appt_date = datetime.date(year, month, day)

    hour = random.choices(
        population=list(range(8, 18)),
        weights=[14,13,12,10,8,6,5,4,3,2]
    )[0]
    appt_time = datetime.time(hour, random.choice([0, 15, 30, 45]))

    status = weighted_status()

    visit_duration = random.choices(
        population=[15, 20, 30, 45, 60],
        weights=[0.20, 0.25, 0.35, 0.15, 0.05]
    )[0] if status == "Completed" else None

    pat_id = random.choice(patient_records)[0]
    dr_id = random.choice(doctor_records)[0]
    appointment_records.append((301+i, appt_date, appt_time, status, visit_duration, pat_id, dr_id))
    if status == "Completed":
        completed_appts.append(301+i)

# Generate Diagnoses
num_diagnoses = len(diagnosis_descs)
diagnosis_codes = [f"D{i:03}" for i in range(1, num_diagnoses + 1)]
diagnosis_records = list(zip(diagnosis_codes, diagnosis_descs))

# Generate Appointment_Diagnosis
appt_diag_records = []
used_pairs = set()

for i in range(num_appt_diag):
    appt_id = random.choice(appointment_records)[0]
    diag_code = random.choice(diagnosis_codes)

    if (appt_id, diag_code) in used_pairs:
        continue

    used_pairs.add((appt_id, diag_code))
    appt_diag_records.append((501+len(appt_diag_records), appt_id, diag_code))


# Generate Billing (ensure unique Appt_ID)
billing_records = []
# Limit billing count to available completed appointments
billing_count = min(num_billing, len(completed_appts))

# Pick unique appointments for billing
unique_appts_for_billing = random.sample(completed_appts, billing_count)

for i, appt_id in enumerate(unique_appts_for_billing):
    total_charge = round(random.uniform(100, 500), 2)
    pay_factor = random.choices([1.0, 0.9, 0.8, 0.5], weights=[0.70, 0.15, 0.10, 0.05])[0]
    amt_paid = round(total_charge * pay_factor, 2)
    payer_fname = random.choice(patient_first_names)
    payer_lname = random.choice(patient_last_names)

    bill_date = datetime.date(2025, random.randint(1, 11), random.randint(1, 28)).strftime('%Y-%m-%d')
    billing_records.append((401+i, appt_id, total_charge, amt_paid, payer_fname, payer_lname, bill_date))
# Write to CSV files
def write_csv(filename, header, records):
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(records)

write_csv("Departments.csv", ["Dept_ID", "Dept_Name"], department_records)
write_csv("Doctors.csv", ["Dr_ID", "Dr_FName", "Dr_LName", "Dept_ID"], doctor_records)
write_csv("Patients.csv", ["Pat_ID", "P_FName", "P_LName", "Pat_DOB", "Pat_Sex", "Pat_Phone", "Pat_Email", "InsurancePlan"], patient_records)
write_csv("Appointments.csv", ["Appt_ID", "Appt_Date", "Appt_Time", "Status", "Visit_Duration", "Pat_ID", "Dr_ID"], appointment_records)
write_csv("Diagnoses.csv", ["Diagnosis_Code", "Diagnosis_Desc"], diagnosis_records)
write_csv("Appointment_Diagnosis.csv", ["ApptDiag_ID", "Appt_ID", "Diagnosis_Code"], appt_diag_records)
write_csv("Billing.csv", ["Bill_ID", "Appt_ID", "Total_Charge", "Amt_Paid", "Payer_FName", "Payer_LName", "Bill_Date"], billing_records)

print("CSV files generated successfully: Departments.csv, Doctors.csv, Patients.csv, Appointments.csv, Diagnoses.csv, Appointment_Diagnosis.csv, Billing.csv")


CSV files generated successfully: Departments.csv, Doctors.csv, Patients.csv, Appointments.csv, Diagnoses.csv, Appointment_Diagnosis.csv, Billing.csv


In [ ]:
from google.colab import files

files.download("Departments.csv")
files.download("Doctors.csv")
files.download("Patients.csv")
files.download("Appointments.csv")
files.download("Diagnoses.csv")
files.download("Appointment_Diagnosis.csv")
files.download("Billing.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>